## Attempt 1 - Accepted

In [ ]:
class Solution:
    def findLadders(self, beginWord: str, endWord: str, wordList: List[str]) -> List[List[str]]:
        sp = dict()
        for w in wordList:
            sp[w] = float('inf')
        
        if endWord not in sp:
            return []
        
        sp[beginWord] = 1
        q = [beginWord]
        while q:
            curr = q.pop()
            L_curr = len(curr)
            for idx in range(L_curr):
                splitted = [d for d in curr]
                for gap in range(26):
                    c = chr(ord('a') + gap)
                    splitted[idx] = c
                    new_word = "".join(splitted)
                    # print(curr, new_word)
                    
                    if sp.get(new_word, -float('inf')) > sp[curr] + 1:
                        sp[new_word] = sp[curr] + 1
                        q.append(new_word)
        
        if sp[endWord] >= float('inf'): ## --> missed this edge case!
            return []
        
        sp1 = [(w, length) for w, length in sp.items()]
        sp1.sort(key=lambda x: x[1])
        print(sp1)
        paths_collection = {beginWord: [[beginWord]]}        
        L = len(sp1)
        for j in range(1, L):
            back = sp1[j][0]
            L_back = len(back)
            curr_collect = []
            for idx in range(L_back):
                splitted = [d for d in back]
                for gap in range(26):
                    c = chr(ord('a') + gap)
                    splitted[idx] = c
                    prev_word = "".join(splitted)
                    if (prev_word in sp) and (sp[prev_word] == sp[back] - 1):
                        curr_collect.extend([p + [back] for p in paths_collection[prev_word]])
            if back == endWord:
                return curr_collect
            paths_collection[back] = curr_collect  

## Attempt 2 - Accepted

In [ ]:
class Solution:
    def findLadders(self, beginWord: str, endWord: str, wordList: List[str]) -> List[List[str]]:
        
        def vary(word):
            res = set([word])
            L = len(word)
            for idx in range(L):
                for j in range(26):
                    new_word = [c for c in word]
                    new_word[idx] = chr(ord('a') + j)
                    res.add("".join(new_word))
            return res
        
        # print(vary('lot'))
        
        wordList = set(wordList) | set([beginWord])
        if endWord not in wordList:
            return []
        
        neighbors = defaultdict(set)
        neighbors[beginWord] = wordList & vary(beginWord)
        for word in wordList:
            neighbors[word] = wordList & vary(word)
        
        # print(neighbors)
        
        if len(neighbors[endWord]) < 1:
            return []
        
        sp = dict()
        sp[beginWord] = 0
        q = [beginWord]
        while q:
            head = q.pop(0)
            for n in neighbors[head]:
                if sp.get(n, float('inf')) > sp[head] + 1:
                    sp[n] = sp[head] + 1
                    q.append(n)
        
        refined = [(cnt, word) for word, cnt in sp.items()]
        refined.sort(key=lambda x: x[0])
        
        res = {beginWord : set([tuple([beginWord])])}
        L = len(refined)
        for j in range(1, L):
            cnt_j, word_j = refined[j]
            curr = set()
            for i in range(j):
                cnt_i, word_i = refined[i]
                if (cnt_i != cnt_j - 1) or (word_j not in neighbors[word_i]):
                    continue
                for path in res[word_i]:
                    to_add = list(path[:]) + [word_j]
                    curr.add(tuple(to_add))
            res[word_j] = curr
            if word_j == endWord:
                return [list(cu) for cu in curr]

## Lessons and Thoughts
- To collect all shortest paths, one way is to sort the `sp` record. Another way is perhaps use BFS again to traverse all paths, and use `sp` to only keep those that are shortest.
- Attempt 2 runs faster though the same algo, probably due to using `set` operations and recording neighbors in the first place.